In [1]:
#First let's create a test script for proof of concept. 
#This is a simple data structure transformation script taken from
#Youtuber 'Real Python' at https://www.youtube.com/watch?v=aysceqdGFw8

#Let's make the proper imports.
import collections
import os
import time
from pprint import pprint

# Serial Processing

The following code will generate a tuple consisting of scientist names and data about them. Then, the tuple is passed through the 'map' process which performs a simple act on each element of the input tuple. The resulting act is saved in a second tuple and then the next entry is processed.

This is performed on one processor on one thread, so the computer must wait for the first entry to finish processing until the second one can begin. 

With the addition of a time.sleep() requirement, this can take quite some time!

In [3]:
Scientist = collections.namedtuple('Scientist', [
    'name',
    'field',
    'born',
    'PhD',
])

scientists = (
    Scientist(name='Sean Lewis', field='Astrophysics', born=1994, PhD=False),
    Scientist(name='Weixiang Yu', field='Astronomy', born=1992, PhD=False),
    Scientist(name='Jaqueline Moreno', field='Astronomy', born=1991, PhD=True),
    Scientist(name='Stephen Sclafani', field='Particle', born=1986, PhD=False),
    Scientist(name='Eli Worth', field='Condensed Matter', born=1993, PhD=False),
    Scientist(name='David Lioi', field='Biophysics', born=1990, PhD=True)
)

pprint(scientists)

def transform(x):
    print('\nProcess ' + str(os.getpid()) + ' working record ' + str(x.name))
    time.sleep(1)
    result = {'name': x.name, 'age': 2020 - x.born}
    print('\nProcess ' + str(os.getpid()) + ' done processing record ' + str(x.name))
    
    return result

start = time.time()


result = tuple(map(
    transform,
    scientists
))

end = time.time()
elapsed = end - start

print('\n')
pprint(result)
print('\n Time to complete:  {0:.3f}'.format(elapsed))

(Scientist(name='Sean Lewis', field='Astrophysics', born=1994, PhD=False),
 Scientist(name='Weixiang Yu', field='Astronomy', born=1992, PhD=False),
 Scientist(name='Jaqueline Moreno', field='Astronomy', born=1991, PhD=True),
 Scientist(name='Stephen Sclafani', field='Particle', born=1986, PhD=False),
 Scientist(name='Eli Worth', field='Condensed Matter', born=1993, PhD=False),
 Scientist(name='David Lioi', field='Biophysics', born=1990, PhD=True))

Process 54720 working record Sean Lewis

Process 54720 done processing record Sean Lewis

Process 54720 working record Weixiang Yu

Process 54720 done processing record Weixiang Yu

Process 54720 working record Jaqueline Moreno

Process 54720 done processing record Jaqueline Moreno

Process 54720 working record Stephen Sclafani

Process 54720 done processing record Stephen Sclafani

Process 54720 working record Eli Worth

Process 54720 done processing record Eli Worth

Process 54720 working record David Lioi

Process 54720 done processing re

Great! We can see the input, which computer core is doing the processing for each task, the output, and the total time to complete the cell. Obviously from the output, the same core is processing each task, and we can watch each task crawl its way through. How can we utilize the full capacity of the quad-core processor in the MacBook Pro?

# Parallel Processing
### Data-based parallel processing

In [4]:
import multiprocessing

pprint(scientists)

start = time.time()
print('\n')

pool = multiprocessing.Pool()
result = pool.map(transform, scientists)

end = time.time()


print('\n')
pprint(result)
print('\n Time to complete:  {0:.3f}'.format(end-start))

(Scientist(name='Sean Lewis', field='Astrophysics', born=1994, PhD=False),
 Scientist(name='Weixiang Yu', field='Astronomy', born=1992, PhD=False),
 Scientist(name='Jaqueline Moreno', field='Astronomy', born=1991, PhD=True),
 Scientist(name='Stephen Sclafani', field='Particle', born=1986, PhD=False),
 Scientist(name='Eli Worth', field='Condensed Matter', born=1993, PhD=False),
 Scientist(name='David Lioi', field='Biophysics', born=1990, PhD=True))



Process 54738 working record Sean Lewis
Process 54741 working record Stephen Sclafani
Process 54739 working record Weixiang Yu
Process 54742 working record Eli Worth
Process 54743 working record David Lioi
Process 54740 working record Jaqueline Moreno






Process 54738 done processing record Sean Lewis
Process 54741 done processing record Stephen Sclafani
Process 54742 done processing record Eli Worth
Process 54743 done processing record David Lioi
Process 54739 done processing record Weixiang Yu
Process 54740 done processing record Jaqu

As you can see, there are 6 different processors that are active in this execution. They each take on one task, this being the transform function on one data table entry and all execute simultaneously. The result is a process that completes ~6x faster than before! 

# And Now I'll Go Even Further Beyond! (Task-Based Parallel Processing)

# www.github.com/seanlabean/PythonOpenMPI.git

## PythonOpenMPI

A generalizable python-mpi utility for task-based parallel programming.

This implementation of task-based parallel programming consists of one root processor, and any number of worker processors. The root breaks a portion of a job into bite sized chunks (like a single file) which are then sent to the workers. While the workers... well... work, the root sits and waits. When a working finishes with its allotted chunk, it pings the root node and asks for another chunk, which the root node then provides. Therefore no worker is ever left without something to do.

This is fundamentally different and more efficient than data-based paralllel processing in which an **entire** job is split into n equally sized chunks (where n is the number of processors) and sent to the worker processors. In this method, when a worker is done processing, it does not need to ask the root node for any more work (since everything has already been distributed to the workers). Therefore, although the task is being completed in parallel, there is a chance that workers will be left idle while they wait for other workers to finish.

## Necessary packages
In addition to whichever packages you use for your process you wish to parallelize, you will need to ensure you have working installs of the following:

* python v3.X
* [OpenMPI v4.X.X](https://www.open-mpi.org/software/ompi/v4.0/)
* mpi4py python library

Ensure you have a properly configured and installed version of OpenMPI with your PATH and PYTHONPATH correctly pointed to the install directory before attempting to `pip install mpi4py`.

## Running the test problem
To run the test problem:
1. `cd` into the `/examples/read_write_parallel` directory.
2. do `python gen_empty_files.py` this will create a sub-directory /files within /read_write_parallel and will populate the directory with 10 blank text files of the form file_X.txt where X runs from 0 to 9. The number of test files created can be changed by editing gen_empty_files.py
3. do `mpiexec -n {num_procs} python test_script.py` while replacing num_procs with the number of processors you wish to parallelize the task across.

The output will reveal the communication between the supervisor processor and the worker processors: transerfing "to-do" data around, sending completed messages back, etc. If you wish, you can edit the chunk size within `test_script.py` to see how the total work is broken up and how the individual processers handle differt sizes of data.

This particular command line call will only parallelize across a single node of any number of processors. If multiple nodes are needed, you will need to provide mpiexec with a hostfile.

## How to parallelize your specific task
Upon investigating the perform_task_in_parallel() funciton within `mpi_utilities.py` you will notice that the "task" can be any function, with any number of input args and kwargs. If your task can be condensed into a single function with a discrete amount of input work, it can be parallelized with this routine. To utilize, call ```perform_task_in_parallel(your_func, [args], {kwargs}, input_data, chunk_size, rank, size, comm, root, debug=False)```
where rank, size, and comm are generated by the initialize_mpi() call. chunk_size is the amount of input work to be sent to each processor when it asks for something to do. [args] and {kwargs} are the arugments and keyword arguments for your_func.

This repository is meant to aid other graduate students in the Drexel University Physics Department, the University at large, and beyond.

Contributors:

- Sean C. Lewis (owner, scl63@drexel.edu)